In [ ]:
#Find Prices for Products (Relies on at least one customer buying one product)
itemAndPrice = list()
for rawItems in itemDatabase["itemRaw"].unique():
    if rawItems.endswith("(x1)"):
        newItemAndPrice = list()
        newItemAndPrice.append(rawItems)
        newItemAndPrice.append(df[(df["transaction_items"] == rawItems)]["transaction_value"].max())
        itemAndPrice.append(newItemAndPrice)
                        
#Align Price to Monthly Orders
alignedPrices = list()
for item in monthlySales["item"]:
    for itemPrices in itemAndPrice:
        itemName = itemPrices[0].split(",")[1]
        if itemName == item:
            alignedPrices.append(itemPrices[1])

#Create Table of Monthly Sales per Item per month
totalSaleValue = monthlySales.copy()
totalSaleValue["price_per_item"] = alignedPrices

#Multiply to product count to get montly sales
for month in uniqueMonths:
    totalSaleValue.iloc[:,month] = totalSaleValue.iloc[:,month].mul(totalSaleValue["price_per_item"], axis=0)
totalSaleValue

In [ ]:
#Change items to lists
def convertList(itemList):
    itemList = list(itemList.split(";"))
    return itemList
df["transaction_items"] = df["transaction_items"].apply(convertList)

#add all items to set to get unique items
uniqueItems = set()
for item in df["transaction_items"]:
    for indivItem in item:
        uniqueItems.add(indivItem)
        
uniqueItems

In [ ]:
#Get Maximum items bought per transaction

maxItem = 0
for item in df["transaction_items"]:
    if len(item) > maxItem:
        maxItem = len(item)
        
maxItem

In [ ]:
#Create new columns for each item per transaction
for i in range(1, maxItem+1):
    df[f"item{i}"] = df["transaction_items"]
    
#Format transaction_items 
def formatItems(itemList):
    while len(itemList) < maxItem:
        itemList.append("")
    return itemList

df["transaction_items"] = df["transaction_items"].apply(formatItems)

#Separate items
def item1separator(itemList):
    return itemList[0]
def item2separator(itemList):
    return itemList[1]
def item3separator(itemList):
    return itemList[2]

#Create new columns for each item per transaction
df["item1"] = df["transaction_items"].apply(item1separator)
df["item2"] = df["transaction_items"].apply(item2separator)
df["item3"] = df["transaction_items"].apply(item3separator)

#Split Lists
def convertList2(itemList):
    itemList = list(itemList.split(","))
    return itemList
for i in range(1, maxItem+1):
    df[f"item{i}"] = df[f"item{i}"].apply(convertList2)

df.head()

In [ ]:
month = 7
userAct = [ 5, 4, 3, 2, 1]
monthlyMetrics = list()
repeaters = 0
inactive = 0
engaged = 0
monthlyMetrics=list()    

if month in userAct:
    pastMonths = set(range(1,month+1))
    if pastMonths == set(userAct):
        engaged+=1
    elif month-1 in userAct:
        repeaters+=1
else:
    pastMonths = list(range(1,month))
    for months in pastMonths:
        if months in userAct:
            inactive +=1
            break

monthlyMetrics.extend([repeaters, inactive, engaged])
monthlyMetrics

In [ ]:
#Create Customer Info Database

customerInfo = pd.DataFrame(columns = ['transaction_date','address', 'birthdate', 'name', 'sex', 'username'])
customerInfo = df[['transaction_date','address', 'birthdate', 'name', 'sex', 'username']]

#Create Record of month and usernames
months = list()
def add_months(givenDate):
    datem = datetime.strptime(givenDate, "%Y/%m/%d")
    months.append(datem.month)
customerInfo["transaction_date"].apply(add_months)

names = list()
def add_names(name):
    names.append(name)
customerInfo["name"].apply(add_names)
             
monthsAndUsername = list(set(zip(months,names)))
mauDf = pd.DataFrame(monthsAndUsername,columns=["month","name"])

#Crete Dataframe for Customer and Months Bought
cmbdf = pd.DataFrame(mauDf["name"].unique(),columns=["name"])
def get_months_bought(name):
    return list(mauDf.loc[mauDf["name"] == name]["month"])
cmbdf["months_bought"] = cmbdf["name"].apply(get_months_bought)

cmbdf

In [ ]:
#Count Total Sales Per Month
monthlyQuant = list()
def total_quant(month):
    monthlyQuant.append(month.sum())
monthlySales.iloc[:,1:len(uniqueMonths)+1].apply(total_quant)
plt.bar(x = monthlySales.iloc[:,1:len(uniqueMonths)+1].columns, height = monthlyQuant)

In [ ]:
#Lola Tamis Test

userAct = [5,4,6]
month = 4
monthlyMetrics = list()
repeaters = 0
inactive = 0
engaged = 0
firstBuyers = 0


if month in userAct:
    pastMonths = set(range(1,month+1))
    if pastMonths == set(userAct) and month != 1:
        engaged+=1
    elif month-1 in userAct:
        repeaters+=1
    else:
        total = 0
        for months in list(range(1,month)):
            if months in userAct:
                total += 1
        if total == 0:
            firstBuyers += 1
else:
    pastMonths = list(range(1,month))
    for months in pastMonths:
        if months in userAct:
            inactive +=1
            break
monthlyMetrics.extend([repeaters, inactive, engaged, firstBuyers])
monthlyMetrics

In [10]:
import pandas as pd

df = pd.DataFrame(['1','2','3','4','5'],columns=["num"])
df["num2"] = ['11','22','33','44','55']
df["nums"] = df["num"] + df["num2"]

df

,num,num2,nums
0,1,11,111
1,2,22,222
2,3,33,333
3,4,44,444
4,5,55,555


In [ ]:
#Create dataframe for items sold per month
uniqueItems = uniqueItemsDf["item_name"]
monthlySales = pd.DataFrame(uniqueItems,columns=["item_name"])
monthlySales["brand"] = uniqueItemsDf["brand"]
uniqueMonths = itemDatabase["date"].unique()

#Count Sales per item per month
for month in uniqueMonths:
    quantsInOneMonth = list()
    for item in uniqueItems:
        quantsInOneMonth.append(itemDatabase[(itemDatabase["date"] == month) & (itemDatabase["item_name"] == item)].sum()["quantity"])
    dummyDate = date(1900, month, 1).strftime('%B')
    monthlySales[dummyDate] = quantsInOneMonth
    
monthlySales.drop(["item_name","brand"], axis = 1).columns

In [ ]:
#Monthly Sales Graph
import matplotlib.pyplot as plt
%matplotlib inline

fig, axes = plt.subplots(6,1)
fig.set_size_inches(20,40)

curGraph = 0
colors = ["Red","Orange","Yellow","Green","Blue","Violet"]
monthlySalesData = monthlySales.drop(["item_name","brand"], axis = 1)
for monthHeader in monthlySalesData.columns:
    axes[curGraph].bar(height=monthlySales[monthHeader], x=monthlySales["item_name"], color=colors[curGraph-1])
    axes[curGraph].set_title(label = f'Product Sales for the Month of {monthHeader}')
    axes[curGraph].set_ylabel("Quantity Sold")
    axes[curGraph].set_xlabel("Product Names")
    axes[curGraph].set_ylim(np.mean(monthlySales[monthHeader])-1000)
    for value in range(len(monthlySalesData[monthHeader])):
        axes[curGraph].annotate(monthlySalesData[monthHeader][value], (monthlySales["item_name"][value], monthlySalesData[monthHeader][value]+15))
    curGraph += 1

In [ ]:
#Find Prices for Products (Relies on at least one customer buying one product)
itemAndPrice = list()
for rawItems in itemDatabase["itemRaw"].unique():
    if rawItems.endswith("(x1)"):
        newItemAndPrice = list()
        newItemAndPrice.append(rawItems)
        newItemAndPrice.append(df[(df["transaction_items"] == rawItems)]["transaction_value"].max())
        itemAndPrice.append(newItemAndPrice)

itemAndPrice = pd.DataFrame(itemAndPrice, columns=["item","price_per_item"])
def get_item(itemName):
    itemName = list(itemName.split(","))[1]
    return itemName
itemAndPrice["item"] = itemAndPrice["item"].apply(get_item)

totalSaleValue = monthlySales.copy()
totalSaleValue = pd.merge(totalSaleValue, itemAndPrice, on="item")

for month in uniqueMonths:
    totalSaleValue.iloc[:,month] = totalSaleValue.iloc[:,month].mul(totalSaleValue["price_per_item"], axis=0)

totalSaleValue